In [1]:
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd
import os

import matplotlib.pyplot as plt
import csv 

In [2]:
try:
    from tqdm import tqdm # long waits are not fun
except:
    print('TQDM does make much nicer wait bars...')
    tqdm = lambda x: x

In [3]:
def make_mask(center,diam,z,width,height,spacing,origin):
    '''
Center : centers of circles px -- list of coordinates x,y,z
diam : diameters of circles px -- diameter
widthXheight : pixel dim of image
spacing = mm/px conversion rate np array x,y,z
origin = x,y,z mm np.array
z = z position of slice in world coordinates mm
    '''
    mask = np.zeros([height,width]) # 0's everywhere except nodule swapping x,y to match img
    #convert to nodule space from world coordinates

    # Defining the voxel range in which the nodule falls
    v_center = (center-origin)/spacing
    v_diam = int(diam/spacing[0]+5)
    v_xmin = np.max([0,int(v_center[0]-v_diam)-5])
    v_xmax = np.min([width-1,int(v_center[0]+v_diam)+5])
    v_ymin = np.max([0,int(v_center[1]-v_diam)-5]) 
    v_ymax = np.min([height-1,int(v_center[1]+v_diam)+5])

    v_xrange = range(v_xmin,v_xmax+1)
    v_yrange = range(v_ymin,v_ymax+1)

    # Convert back to world coordinates for distance calculation
    x_data = [x*spacing[0]+origin[0] for x in range(width)]
    y_data = [x*spacing[1]+origin[1] for x in range(height)]

    # Fill in 1 within sphere around nodule
    for v_x in v_xrange:
        for v_y in v_yrange:
            p_x = spacing[0]*v_x + origin[0]
            p_y = spacing[1]*v_y + origin[1]
            if np.linalg.norm(center-np.array([p_x,p_y,z]))<=diam:
                mask[int((p_y-origin[1])/spacing[1]),int((p_x-origin[0])/spacing[0])] = 1.0
    return(mask)



'''masks = np.ndarray([3,height,width],dtype=np.uint8)
mask = make_mask(center, diam, i_z*spacing[2]+origin[2],width, height, spacing, origin)
fig,ax = plt.subplots(2,2,figsize = [8,8])
    ax[0,0].imshow(imgs[i,:,:],cmap = 'gray')
    ax[0,1].imshow(masks[i,:,:],cmap = 'gray')
plt.show()'''

"masks = np.ndarray([3,height,width],dtype=np.uint8)\nmask = make_mask(center, diam, i_z*spacing[2]+origin[2],width, height, spacing, origin)\nfig,ax = plt.subplots(2,2,figsize = [8,8])\n    ax[0,0].imshow(imgs[i,:,:],cmap = 'gray')\n    ax[0,1].imshow(masks[i,:,:],cmap = 'gray')\nplt.show()"

In [4]:
def matrix2int16(matrix):
    ''' 
matrix must be a numpy array NXN
Returns uint16 version
    '''
    m_min= np.min(matrix)
    m_max= np.max(matrix)
    matrix = matrix-m_min
    return(np.array(np.rint( (matrix-m_min)/float(m_max-m_min) * 65535.0),dtype=np.uint16))

In [5]:
def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)

In [6]:
import cv2
from skimage.transform import pyramid_reduce, resize

if __name__ == '__main__':
    # Getting list of image files
    
    luna_path = "D:/Lungcancer/LUNA/"
    luna_scratch_path = "D:/Lungcancer/LUNA/"
    luna_subset_path = luna_scratch_path+"subset9/"
    output_path = "D:/Lungcancer/LUNA/3D_Unet/nptest/"
    file_list=glob(luna_subset_path+"*.mhd")
    # The locations of the nodes
    df_node = pd.read_csv(luna_path+"annotations.csv")
    df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))
    df_node = df_node.dropna()

    print(len(file_list))

    #####
    #
    # Looping over the image files
    #
    allPatientMask = []
    allPatientImg = []

    endnum = 0
    for fcount, img_file in enumerate(tqdm(file_list)):
        print("#"*80)
        if endnum==50:
            continue
        endnum+=1
        mini_df = df_node[df_node["file"]==img_file] #get all nodules associate with file
        print('---'*30)
        print(mini_df)
        print('---'*30)
        personImg = sitk.ReadImage(img_file)
        personImg_array = sitk.GetArrayFromImage(personImg)
        print(personImg_array.shape)
        print(len(personImg_array))


        resizePersonImg = np.ndarray([len(personImg_array),256,256,1],dtype=np.float32)
        for i in range(len(personImg_array)):
            resizePersonImg[i] = resize(personImg_array[i], output_shape=(256, 256,1), preserve_range=True)
            for j in range(resizePersonImg[i].shape[0]):
                        for k in range(resizePersonImg[i].shape[1]):
                            if resizePersonImg[i][j,k]<=-2048:
                                resizePersonImg[i][j,k]=0


        personImg_mask = np.ndarray([len(personImg_array),256,256,1],dtype=np.float32)
        for i in range(len(personImg_array)):
            personImg_mask[i] = np.zeros([256,256,1])

        print(personImg_array.shape)
        #allPatientImg = personImg_array[]
        '''
        for i in range(len(personImg_array)):
            #personImg_imgs[i] = personImg_array[i]
            fig = plt.figure()
            plt.imshow(personImg_array[i,:,:], cmap='gray') #顯示所有圖片(0軸)
            plt.axis('off')
            plt.savefig('D:/Lungcancer/LUNA/3D_Unet/img_test/'+str(fcount)+'_'+str(i)+'.png', dpi=fig.dpi, bbox_inches='tight', pad_inches = 0)
            plt.close('all')

            personImg_mask[i] = np.zeros([personImg_array.shape[1],personImg_array.shape[2]])
            maskfig = plt.figure()
            plt.imshow(personImg_mask[i,:,:], cmap='gray') #顯示所有圖片(0軸)
            plt.axis('off')
            plt.savefig('D:/Lungcancer/LUNA/3D_Unet/mask_test/'+str(fcount)+'_'+str(i)+'.png', dpi=fig.dpi, bbox_inches='tight', pad_inches = 0)
            plt.close('all')'''
            #cv2.imwrite('D:/Lungcancer/LUNA/3D_Unet/test/'+str(i)+'.png',printAllImg_imgs[i,:,:])
        if mini_df.shape[0]>0: # some files may not have a nodule--skipping those 
            # load the data once
            itk_img = sitk.ReadImage(img_file) 
            img_array = sitk.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
            num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
            origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
            spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
            # go through all nodes (why just the biggest?)
            for node_idx, cur_row in mini_df.iterrows():       
                node_x = cur_row["coordX"]
                node_y = cur_row["coordY"]
                node_z = cur_row["coordZ"]
                diam = cur_row["diameter_mm"]


                center = np.array([node_x, node_y, node_z])   # nodule center
                print('center: '+str(center))
                v_center = np.rint((center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering) #實際座標
                print('v_center: '+str(v_center))

                diam_range = int(round(diam/spacing[0]/spacing[2] , 1))
                print('diam_range: '+str(diam_range))
                imgs = np.ndarray([diam_range+1,height,width],dtype=np.float32)
                masks = np.ndarray([diam_range+1,height,width],dtype=np.uint8)

                diam = diam-spacing[2]*diam_range/2
                isExtendRange = diam_range%2
                for i, i_z in enumerate(np.arange(int(v_center[2])-int(diam_range/2+isExtendRange),
                                 int(v_center[2])+int(diam_range/2 + isExtendRange)).clip(0, num_z-1)): # clip prevents going out of bounds in Z  #clip(0,5) -1<0 = 0 , 6>5 = 5
                    '''print('1*'*20)
                    print(i_z)
                    print('2*'*20)
                    print(spacing)
                    print('3*'*20)
                    print(origin)
                    print('4*'*20)
                    print(width)
                    print('5*'*20)
                    print(height)
                    print('6*'*20)
                    print(diam)'''
                    mask = make_mask(center, diam, i_z*spacing[2]+origin[2],width, height, spacing, origin)
                    ###
                    if i_z < int(v_center[2]):
                        diam += spacing[2]
                    else:
                        diam -= spacing[2]
                    ###
                    personImg_mask[i_z] = resize(mask, output_shape=(256, 256,1), preserve_range=True)

                    #imgsmasks[i] = masks[i]*imgs[i]
                    '''plt.imshow(masks[i,:,:], cmap='gray') #顯示所有圖片(0軸)
                    plt.axis('off')
                    plt.savefig('D:/Lungcancer/LUNA/3D_Unet/mask_test/'+str(fcount)+'_'+str(i_z)+'.png', dpi=fig.dpi, bbox_inches='tight', pad_inches = 0)
                    plt.close('all')'''
                    '''fig,ax = plt.subplots(2,2,figsize = [8,8])
                    ax[0,0].imshow(resizePersonImg[i_z,:,:,0],cmap = 'gray')
                    ax[0,1].imshow(personImg_mask[i_z,:,:,0],cmap = 'gray')
                    ax[1,0].imshow(resizePersonImg[i_z,:,:,0]*personImg_mask[i_z,:,:,0],cmap = 'gray')
                    plt.show()'''
        
            if len(personImg_array)>=128:
                finalPatientImg = resizePersonImg[int(len(personImg_array)/2)-64:int(len(personImg_array)/2)+64]
                allPatientImg.append(finalPatientImg)
                finalPatientMask = personImg_mask[int(len(personImg_array)/2)-64:int(len(personImg_array)/2)+64]
                allPatientMask.append(finalPatientMask)
            print(finalPatientImg.shape)
    
    allPatientImg = np.array(allPatientImg)
    allPatientMask = np.array(allPatientMask)


    print('this : '+str(allPatientImg.shape))
    for i in range(len(allPatientImg)):
        print(np.array(allPatientImg[i]).shape)

    np.save(os.path.join(output_path,"threeDimages.npy") ,allPatientImg)
    np.save(os.path.join(output_path,"threeDmasks.npy" ),allPatientMask)
    #np.save(os.path.join(output_path,"imgsmasks_%04d_%04d.npy" % (fcount, node_idx)),imgsmasks)

88


  0%|          | 0/88 [00:00<?, ?it/s]

################################################################################
------------------------------------------------------------------------------------------
                                           seriesuid      coordX    coordY  \
8  1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...  105.055792  19.82526   

      coordZ  diameter_mm                                               file  
8 -91.247251    21.089619  D:/Lungcancer/LUNA/subset9\1.3.6.1.4.1.14519.5...  
------------------------------------------------------------------------------------------
(143, 512, 512)
143
(143, 512, 512)
center: [105.0557924   19.82526014 -91.24725078]
v_center: [354. 279. 121.]
diam_range: 9


  1%|          | 1/88 [00:13<19:03, 13.14s/it]

(128, 256, 256, 1)
################################################################################
------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [seriesuid, coordX, coordY, coordZ, diameter_mm, file]
Index: []
------------------------------------------------------------------------------------------
(468, 512, 512)
468


...

 56%|█████▌    | 49/88 [20:51<21:22, 32.90s/it]

(471, 512, 512)
################################################################################
------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [seriesuid, coordX, coordY, coordZ, diameter_mm, file]
Index: []
------------------------------------------------------------------------------------------
(144, 512, 512)
144


100%|██████████| 88/88 [21:04<00:00, 14.37s/it]

(144, 512, 512)
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
############

this : (32, 128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
(128, 256, 256, 1)
